<a href="https://colab.research.google.com/github/MustofAhmed41/3D-Jeep-Car/blob/main/FL_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
from sklearn.utils import shuffle
import pandas as pd

import random

In [2]:
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
from tensorflow.keras.datasets import mnist

import numpy as np

# Load the MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Create a list to hold the data for each client
clients_data = [[] for _ in range(10)]

# Shuffle the dataset
indices = np.random.permutation(len(train_images))
shuffled_images = train_images[indices]
shuffled_labels = train_labels[indices]

# Partition the data equally among 30 clients
for i in range(len(shuffled_images)):
    label = shuffled_labels[i]
    client_idx = i % 10  # Distribute evenly among 50 clients
    clients_data[client_idx].append((shuffled_images[i], label))

# Convert the list of lists to numpy arrays
clients_data = [np.array(client_data) for client_data in clients_data]

# Verify that all clients have an equal number of labels for all 10 digits
for i, client_data in enumerate(clients_data):
    labels, counts = np.unique(client_data[:, 1], return_counts=True)
    print(f"Client {i} label distribution: {dict(zip(labels, counts))}")


11490434/11490434 [==============================] - 0s 0us/step


<ipython-input-3-f34bc6f25370>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  clients_data = [np.array(client_data) for client_data in clients_data]


Client 0 label distribution: {0: 610, 1: 664, 2: 611, 3: 594, 4: 564, 5: 536, 6: 591, 7: 621, 8: 597, 9: 612}
Client 1 label distribution: {0: 612, 1: 684, 2: 583, 3: 601, 4: 635, 5: 503, 6: 584, 7: 646, 8: 578, 9: 574}
Client 2 label distribution: {0: 612, 1: 727, 2: 551, 3: 638, 4: 573, 5: 509, 6: 606, 7: 603, 8: 584, 9: 597}
Client 3 label distribution: {0: 596, 1: 700, 2: 606, 3: 602, 4: 578, 5: 562, 6: 597, 7: 605, 8: 586, 9: 568}
Client 4 label distribution: {0: 570, 1: 648, 2: 577, 3: 627, 4: 572, 5: 546, 6: 602, 7: 665, 8: 605, 9: 588}
Client 5 label distribution: {0: 582, 1: 666, 2: 586, 3: 621, 4: 588, 5: 537, 6: 555, 7: 646, 8: 577, 9: 642}
Client 6 label distribution: {0: 572, 1: 653, 2: 581, 3: 607, 4: 591, 5: 602, 6: 588, 7: 627, 8: 616, 9: 563}
Client 7 label distribution: {0: 573, 1: 643, 2: 640, 3: 639, 4: 591, 5: 527, 6: 588, 7: 621, 8: 560, 9: 618}
Client 8 label distribution: {0: 620, 1: 675, 2: 609, 3: 603, 4: 558, 5: 534, 6: 602, 7: 603, 8: 582, 9: 614}
Client 9 l

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [5]:
layers_shape = []

In [7]:
for j,client_data in enumerate(clients_data):
  # Split the data into features (images) and labels
  client_images, client_labels = list(client_data[:, 0]), list(client_data[:, 1])
  # Normalize the image data
  client_images = np.array(client_images) / 255.0

  # Convert the labels to one-hot encoding
  client_labels = tf.keras.utils.to_categorical(client_labels, 10)

  # Create a TensorFlow Dataset from the NumPy arrays
  dataset = tf.data.Dataset.from_tensor_slices((client_images, client_labels)).shuffle(len(client_images)).batch(32)

  # Define the model architecture
  model = models.Sequential()
  model.add(layers.Flatten(input_shape=(28, 28)))
  model.add(layers.Dense(256,  activation='relu', name="layer_1"))
  model.add(layers.Dense(512, activation='relu', name="layer_2"))
  model.add(layers.Dense(10, activation='softmax', name="layer_3"))

  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Train the model using the TensorFlow Dataset
  model.fit(dataset, epochs=2)

  for layer in model.layers:
    if hasattr(layer, 'weights'):
      weights = layer.get_weights()
      for i, weight in enumerate(weights):
        file_name = "model_" + str(j) + "_" + str(layer.name)
        if i == 0:
          file_name = file_name + "_weights.csv"
          # print(file_name)
          np.savetxt(file_name, weight, delimiter=',')
        else:
          file_name = file_name + "_biases.csv"
          # print(file_name)
          np.savetxt(file_name, weight, delimiter=',')

  if j == len(clients_data)-1:
    for layer in model.layers:
      if hasattr(layer, 'weights'):
        weights = layer.get_weights()
        temp = []
        for i, weight in enumerate(weights):
          temp.append(weight.shape)

        layers_shape.append(temp)

Epoch 1/2
188/188 [==============================] - 6s 14ms/step - loss: 0.5150 - accuracy: 0.8492
Epoch 2/2
188/188 [==============================] - 1s 8ms/step - loss: 0.2013 - accuracy: 0.9373
Epoch 1/2
188/188 [==============================] - 2s 7ms/step - loss: 0.5204 - accuracy: 0.8458
Epoch 2/2
188/188 [==============================] - 2s 11ms/step - loss: 0.2055 - accuracy: 0.9380
Epoch 1/2
188/188 [==============================] - 2s 8ms/step - loss: 0.4884 - accuracy: 0.8563
Epoch 2/2
188/188 [==============================] - 1s 7ms/step - loss: 0.1778 - accuracy: 0.9478
Epoch 1/2
188/188 [==============================] - 2s 7ms/step - loss: 0.5395 - accuracy: 0.8415
Epoch 2/2
188/188 [==============================] - 2s 11ms/step - loss: 0.2170 - accuracy: 0.9337
Epoch 1/2
188/188 [==============================] - 2s 7ms/step - loss: 0.5171 - accuracy: 0.8467
Epoch 2/2
188/188 [==============================] - 1s 7ms/step - loss: 0.2045 - accuracy: 0.9363
Epoch 1

In [57]:
def get_model_weights(layer_name):
  weight_file_name = layer_name + "_weights.csv"
  bias_file_name = layer_name + "_biases.csv"
  weight = my_array = np.loadtxt(weight_file_name, delimiter=',')
  biases = my_array = np.loadtxt(bias_file_name, delimiter=',')
  return [weight, biases]

In [58]:
def fed_avg():
  for i,layer_name in enumerate(layer_names_lst):
    temp = np.zeros((layers_shape[i][0][0], layers_shape[i][0][1]))
    for j, model_name in enumerate(model_names_lst):
      file_name  = model_name + layer_name + weights_biases[0]
      temp = temp + np.loadtxt(file_name, dtype=np.float64, delimiter=",")
    temp = temp / len(clients_data)
    save_file_name = 'aggregated_' + layer_name + weights_biases[0]
    np.savetxt(save_file_name, temp, delimiter=',')

  for i,layer_name in enumerate(layer_names_lst):
    temp = np.zeros((layers_shape[i][1][0]))
    for j, model_name in enumerate(model_names_lst):
      file_name  = model_name + layer_name + weights_biases[1]
      temp = temp + np.loadtxt(file_name, dtype=np.float64, delimiter=",")
    temp = temp / len(clients_data)
    save_file_name = 'aggregated_' + layer_name + weights_biases[1]
    np.savetxt(save_file_name, temp, delimiter=',')

In [8]:
def aggregated_model():
  latest_model = keras.models.Sequential()
  latest_model.add(keras.layers.Dense(50, weights=get_model_weights("aggregated_layer_1"),  activation='relu',  input_shape=(784,), name="layer_1"))
  latest_model.add(keras.layers.Dense(20, weights=get_model_weights("aggregated_layer_2"),  activation='relu',  name="layer_2"))
  latest_model.add(keras.layers.Dense(num_classes, weights=get_model_weights("aggregated_layer_3"), activation='softmax',  name="layer_3"))

  loss = keras.losses.SparseCategoricalCrossentropy()
  optim = keras.optimizers.Adam(learning_rate=0.001)
  metrics = [keras.metrics.SparseCategoricalAccuracy()]

  latest_model.compile(loss=loss, optimizer=optim, metrics=metrics)

  print("Evaluate:")
  x_train, y_train, x_test, y_test = create_dataset()

  latest_model.evaluate(x_test,  y_test, verbose=2)



In [11]:
layer_names_lst = ["layer_1_", "layer_2_", "layer_3_"]
model_names_lst = []
weights_biases = ["weights.csv", "biases.csv"]

for i in range(1, len(clients_data)+1):
  model = "model_" + str(i) + "_"
  model_names_lst.append(model)

print(layer_names_lst)
print(model_names_lst)

['layer_1_', 'layer_2_', 'layer_3_']
['model_1_', 'model_2_', 'model_3_', 'model_4_', 'model_5_', 'model_6_', 'model_7_', 'model_8_', 'model_9_', 'model_10_']
layer_1_
layer_2_
layer_3_


In [62]:
fed_avg()

IndexError: ignored